In [2]:
from FlagEmbedding import FlagModel

model = FlagModel("BAAI/bge-large-en", 
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages: "
    )

In [3]:
queries = ['conflicting', 'inexperience', 'assumption', 'appreciation', 'genetics', 'client relations']

In [4]:
import pandas as pd

consol_pre = pd.read_csv('./data/pre_consolidated.csv')
sep_pre = consol_pre[['ID'] + list(consol_pre.columns[14:-2])]
sep_pre_col_names = ['ID', 'Q14', 'Q15', 'Q16', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25']
sep_pre.columns = sep_pre_col_names

consol_post = pd.read_csv('./data/post_consolidated.csv')
sep_post = consol_post[['ID'] + list(consol_post.columns[14:-3])]
sep_post_col_names = ['ID', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26']
sep_post.columns = sep_post_col_names

pre_fr = sep_pre[['ID', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25']]
post_fr = sep_post[['ID', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26']]

In [12]:
from nltk import sent_tokenize

all_responses = []
for col in pre_fr.columns[1:]:
    pre_sent = pre_fr[col].apply(sent_tokenize)
    pre_sent = pre_sent.explode()
    for fr in list(pre_sent):
        all_responses.append(fr)

for col in post_fr.columns[1:]:
    post_sent = post_fr[col].apply(sent_tokenize)
    post_sent = post_sent.explode()
    for fr in list(post_sent):
        all_responses.append(fr)

len(all_responses)

253

In [13]:
q_embeddings = model.encode_queries(queries)
p_embeddings = model.encode(all_responses)

In [14]:
scores = q_embeddings @ p_embeddings.T

In [16]:
scores.shape

(6, 253)

In [35]:
scores[1].argsort()[::-1]

array([ 68,  74, 217, 229, 170,  51,  48, 206, 112,  85, 214,  66,  97,
       196, 209, 189, 228,   4,  94,  99, 220, 225, 208,  67,  11, 103,
       105, 222, 249,  81,  18,  70, 100,  96, 137, 125,   3, 211,  79,
       117, 161, 210, 204, 102,  31,  84, 111, 124,  15,  13, 190, 179,
       219, 166, 114, 139, 223, 216, 176,   5,  80,  75, 213,   0,  14,
        95,  88,  10, 231, 212,  65,  83,  22, 230,  53,  82, 226,   9,
       118, 250, 104, 101, 175,  93,  71, 158,   6,  77,  98,   7,  69,
       129, 198,  73,  54,  58,  12, 224,  63, 147, 136,  90, 168,  72,
       153, 113, 133, 164,  44, 163,  64,  91, 171,  24, 169, 199, 108,
       201, 246, 157,   1, 132, 110,  20, 187, 251,  52, 123,  92, 120,
        28,  86,  26, 215,  45, 119, 221,  43,  42,  47,  56,  39, 148,
       240, 151, 109,  38,  32, 202, 185, 154,  78, 116, 155, 200, 126,
       156,  30,  89, 245, 107, 197,  60, 182,  76, 106, 150,   8,  57,
       146, 130, 173, 135,  41, 165, 167, 252, 241, 178, 247, 12

In [36]:
i = 68
all_responses[i], scores[0][i]

('I have very little experience with canine generics and theriogenology.',
 0.70914245)